In [5]:
%pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.5 MB 7.4 MB/s 


In [6]:
import pandas as pd

# Origem dos dados
url = '2DadosParaDesafioOtimizacaoTransporte.xlsx'

#Leitura dos dados
data_pedidos = pd.read_excel(url, sheet_name = "Pedido")
data_produtos = pd.read_excel(url, sheet_name = "Produto")
data_veiculos = pd.read_excel(url, sheet_name = "Veículos")
data_destinatarios = pd.read_excel(url, sheet_name = "Destinatário")
data_origem = pd.read_excel(url, sheet_name = "Origem")

# Preparo dos dados
features = ['Código', 'Produto', 'Destinatário', 'Quantidade']
pedidos = data_pedidos[features]

features = ['Código', 'Disponível', 'Peso (kg)', 'Volume (m³)', 'Altura', 'Largura', 'Comprimento']
produtos = data_produtos[features]

features = ['Placa', 'Latitude', 'Longitude', 'Lotação (kg)', 'Cubagem (m³)', 'Altura', 'Largura', 'Comprimento', 'Custo / Km', 'Tempo de trabalho (horas)']
veiculos = data_veiculos[features]

features = ['Código', 'Latitude', 'Longitude', '*início recebimento', '*Fim recebimento']
destinatarios = data_destinatarios[features]

features = ['Código', 'Latitude', 'Longitude']
origens = data_origem[features]

print('Leitura Concluída')

# Agrupamento dos pedidos por destinatários
agrupado = pedidos.groupby('Destinatário')

volume_destinatario = []
peso_destinatario = []
lista_destinatarios = []

for d, p in agrupado:
       
    volume_total = 0
    peso_total = 0
    
    lista_destinatarios.append(d)
    
    for PRD in p.Produto:

        soma = float(produtos.loc[produtos['Código'] == PRD]['Volume (m³)']) * float(sum((p.loc[p['Produto'] == PRD]['Quantidade'])))
        volume_total = volume_total + soma
        
        soma = float(produtos.loc[produtos['Código'] == PRD]['Peso (kg)']) * float(sum((p.loc[p['Produto'] == PRD]['Quantidade'])))
        peso_total = peso_total + soma
    
    volume_destinatario.append(volume_total)
    peso_destinatario.append(peso_total)
print('Separação dos dados concluída com sucesso')

new_volume_destinatario = []
new_peso_destinatario = []
new_lista_destinatarios = []
for i in destinatarios.Código:
    if i in lista_destinatarios:
        new_volume_destinatario.append(volume_destinatario[lista_destinatarios.index(i)])
        new_peso_destinatario.append(peso_destinatario[lista_destinatarios.index(i)])
        new_lista_destinatarios.append(i)
print("Redução concluída com sucesso")

# Constantes para empacotamento
peso_produtos = {produto: peso for produto, peso in zip(new_lista_destinatarios, new_peso_destinatario)}
volume_produtos = {produto: volume for produto, volume in zip(new_lista_destinatarios, new_volume_destinatario)}

capacidade_veiculos = {veiculo: capacidade for veiculo, capacidade in zip(veiculos['Placa'], veiculos['Lotação (kg)'])}
area_veiculos = {veiculo: area for veiculo, area in zip(veiculos['Placa'], veiculos['Cubagem (m³)'])}
custo_veiculos = {veiculo: custo for veiculo, custo in zip(veiculos['Placa'], veiculos['Custo / Km'])}

print('Constantes de empacotamento: OK')
  
coordenadas = destinatarios[['Latitude', 'Longitude']]
coordenadas.index = destinatarios.Código

origem = pd.DataFrame([[origens.Latitude, origens.Longitude]], 
                        columns=['Latitude', 'Longitude'], index=origens.Código)

new_lista_destinatarios2 = new_lista_destinatarios.copy()
for i in origens.Código:
    new_lista_destinatarios.append(i)

coordenadas = coordenadas.append(origem)

print('Lista de Destinatários e Coordenadas: OK')

# Libs adicionais
import math
from itertools import product

# Distância Euclidiana entre cada pár de vértice
def distance(vertice1, vertice2):
    if(vertice1 == vertice2):
        return 999999
    v1 = coordenadas.loc[vertice1]
    v2 = coordenadas.loc[vertice2]
    diff = (v1[0]-v2[0], v1[1]-v2[1])
    return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])

dist = {(v1, v2): distance(v1, v2) for v1, v2 in product(new_lista_destinatarios, repeat=2)}

print('Distâncias: OK')


FileNotFoundError: ignored

In [ ]:
import gurobipy as gp
from gurobipy import GRB

model = gp.Model()

# Variável X = Qual a rota de cada veículo
x = model.addVars(new_lista_destinatarios, new_lista_destinatarios, veiculos['Placa'], vtype=GRB.BINARY, name='x')
# Variável Y = Qual veículo foi escolhido
y = model.addVars(veiculos['Placa'], vtype=GRB.BINARY, name='y')
# Variável auxiliar para a eliminação de subrotas
u = model.addVars(new_lista_destinatarios, vtype=GRB.CONTINUOUS, name='u')

# Função Objetivo: Maximizar o número de itens transportados
model.setObjective(
    gp.quicksum(dist[(i, j)] * custo_veiculos[k] * x[i, j, k] 
                                        for i in new_lista_destinatarios
                                        for j in new_lista_destinatarios
                                        for k in veiculos['Placa']),
    sense=GRB.MINIMIZE)

# ----------------------------- RESTRIÇÕES DE ROTEAMENTO -----------------------------

# Restrição 1: Garante que todos os destinatários sejam visitados pelo menos uma vez
r1 = model.addConstrs(
    gp.quicksum(x[i, j, k] for i in new_lista_destinatarios for k in veiculos['Placa']) == 1
                           for j in new_lista_destinatarios if (i!=j))

# Restrição 2: Conservação de fluxo
r2 = model.addConstrs(
    gp.quicksum(x[i, l, k] for i in new_lista_destinatarios if (i!=l)) - 
    gp.quicksum(x[l, j, k] for j in new_lista_destinatarios if (l!=j)) == 0
                           for l in new_lista_destinatarios
                           for k in veiculos['Placa'])

# Restrição 3: A origem deve estar presente em todos os veículos que forem selecionados
r3 = model.addConstrs(
    gp.quicksum(x['ORI-001', j, k] for j in new_lista_destinatarios) == y[k]
                                   for k in veiculos['Placa'])

# Restrição 4: Restrição de eliminação de subrotas
r5 = model.addConstrs(u[i] - u[j] + 
                      len(new_lista_destinatarios) * gp.quicksum(x[i, j, k] for k in veiculos['Placa']) <= 
                      len(new_lista_destinatarios2) 
                          for i in new_lista_destinatarios2
                          for j in new_lista_destinatarios2
                          if (i != j))


#---------------------------------EMPACOTAMENTO---------------------------------
# Restrição 1: O somatório dos pesos deve ser menor que a capacidade do veiculo
e1 = model.addConstrs(
    gp.quicksum(x[i, j, k] * peso_produtos[i]  for i in new_lista_destinatarios2 for j in new_lista_destinatarios) <= 
                capacidade_veiculos[k] for k in veiculos['Placa'])

# Restrição 2: O somatório dos volumes deve ser menor que a area do veículo
e2 = model.addConstrs(
    gp.quicksum(x[i, j, k] * volume_produtos[i] for i in new_lista_destinatarios2 for j in new_lista_destinatarios) <= 
                area_veiculos[k] for k in veiculos['Placa'])

print('ok')

ok


In [ ]:
def NC_Callback(M,where):
   if where == gp.GRB.Callback.MIP:
      tempo = M.cbGet(gp.GRB.Callback.RUNTIME)
      if tempo > 60:
         M.terminate( )
        
model.Params.lazyConstraints = 1
model.optimize(NC_Callback)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1071 rows, 4841 columns and 29655 nonzeros
Model fingerprint: 0x80d404b9
Variable types: 31 continuous, 4810 integer (4810 binary)
Coefficient statistics:
  Matrix range     [1e-01, 3e+03]
  Objective range  [3e+00, 5e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+04]
Found heuristic solution: objective 1.268999e+08
Presolve removed 5 rows and 6 columns
Presolve time: 0.04s
Presolved: 1066 rows, 4835 columns, 24975 nonzeros
Variable types: 31 continuous, 4804 integer (4804 binary)
Found heuristic solution: objective 1.237499e+08

Root relaxation: objective 3.182180e+02, 472 iterations, 0.02 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0  

In [ ]:
# Valor objetivo
print("Valor Total:", model.objVal)

# Mostra os itens nas mochilas
resultado = 0
lista_de_destinatarios = []
for k in veiculos['Placa']:
    print('----------------------')
    print(k)
    print('----------------------', resultado, '----------------------------')
    resultado = resultado + 1
    lista = []
    for i in new_lista_destinatarios:
        for j in new_lista_destinatarios:         
            if x[i, j, k].x != 0:
                print(i, j)
                lista.append(i)
    print()
    lista_de_destinatarios.append(lista)

Valor Total: 20700533.90331809
----------------------
RRS-7812
---------------------- 0 ----------------------------
DES-001 DES-019
DES-004 DES-006
DES-005 DES-018
DES-006 DES-016
DES-007 DES-009
DES-008 DES-030
DES-009 DES-012
DES-010 DES-023
DES-011 DES-007
DES-012 DES-014
DES-014 DES-008
DES-016 DES-020
DES-017 DES-011
DES-018 DES-004
DES-019 DES-024
DES-020 DES-027
DES-023 DES-005
DES-024 DES-025
DES-025 DES-028
DES-026 DES-010
DES-027 ORI-001
DES-028 DES-029
DES-029 DES-026
DES-030 DES-001
ORI-001 DES-017

----------------------
WGQ-1382
---------------------- 1 ----------------------------
DES-003 DES-003
DES-021 DES-021

----------------------
ZSN-3341
---------------------- 2 ----------------------------

----------------------
XCW-1633
---------------------- 3 ----------------------------
DES-013 DES-013
DES-015 DES-015

----------------------
JMN-4177
---------------------- 4 ----------------------------
DES-002 DES-002
DES-022 DES-022



In [ ]:
# Calcula os valores
lista_soma_peso = []
lista_soma_volume = []
for i in lista_de_destinatarios:
    soma_peso = 0
    soma_volume = 0
    for j in i:
        if j!='ORI-001':
            soma_peso = soma_peso + peso_produtos[j]
            soma_volume = soma_volume + volume_produtos[j]
    lista_soma_peso.append(soma_peso)
    lista_soma_volume.append(soma_volume)

In [ ]:
# Imprimir resultado
cont = 0
for i, j in enumerate(veiculos['Placa']):
    cont = cont + 1
    print('', cont,'-', j)
    print('-----------')
    print('Soma do Peso: ', lista_soma_peso[i])
    print('Capacidade do veículo: ', capacidade_veiculos[j])
    print('Soma do Volume: ', lista_soma_volume[i])
    print('Área do veículo: ', area_veiculos[j])
    print()

 1 - RRS-7812
-----------
Soma do Peso:  32878.0
Capacidade do veículo:  33000
Soma do Volume:  23.839898
Área do veículo:  385.0

 2 - WGQ-1382
-----------
Soma do Peso:  5565.0
Capacidade do veículo:  6000
Soma do Volume:  14.728697
Área do veículo:  160.16

 3 - ZSN-3341
-----------
Soma do Peso:  0
Capacidade do veículo:  40000
Soma do Volume:  0
Área do veículo:  398.75

 4 - XCW-1633
-----------
Soma do Peso:  3578.0
Capacidade do veículo:  6000
Soma do Volume:  7.344608
Área do veículo:  160.16

 5 - JMN-4177
-----------
Soma do Peso:  4404.0
Capacidade do veículo:  6000
Soma do Volume:  2.9466910000000004
Área do veículo:  160.16

